In [2]:
#librerias
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy import inspect

In [3]:
#Conectar el motor al archivo de la base de datos transalacional y WH
engine = create_engine('sqlite:///chinook.db')
engineWH = create_engine('sqlite:///guia.db')

# Extract

In [4]:
# SQL Expression Language crea metadatos que contienen objetos que definen la tabla de clientes
metadata = MetaData()

# Este método instancia las tablas que ya
# existe en la base de datos, a la que está conectado el motor.
metadata.create_all(engine)
metadata.create_all(engineWH)
# Revisando esto, podemos ver la estructura de la tabla y los tipos de variables 
inspector = inspect(engine)
inspectorWH = inspect(engineWH)

#Tablas de db tranlacinal chinook - invoice_items
#print('Chinook-invoice_items')
#print(inspector.get_columns('invoice_items'))
#Tablas de echo - fact_sales
#print('\n WH - Fact_sales')
#print(inspectorWH.get_columns('Fact_sales'))

In [4]:
#datos de db
inspector.get_columns('employees')
inspector.get_columns('albums')
inspector.get_columns('media_types')
inspector.get_columns('genres')
inspector.get_columns('playlists')
inspector.get_columns('artists')
inspector.get_columns('invoices')
inspector.get_columns('customers')
inspector.get_columns('tracks')


[{'name': 'TrackId',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 1},
 {'name': 'Name',
  'type': NVARCHAR(length=200),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'AlbumId',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'MediaTypeId',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'GenreId',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'Composer',
  'type': NVARCHAR(length=220),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'Milliseconds',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'Bytes',
  'type': INTEGER(),
  'nullable': True,
  'd

# Transform

In [8]:
import pandas as pd
# dim_employees
df_employees=pd.read_sql_query("""SELECT LastName, FirstName, Title, ReportsTo, BirthDate, HireDate, Address, 
City, State, Country, PostalCode, Phone, Fax, Email
FROM employees;
 """, con=engine.connect())
df_employees.head()
#dim_albums
df_albums=pd.read_sql_query("""SELECT AlbumId, Title
FROM albums;
 """, con=engine.connect())
df_albums.head()
#dim_media_types
df_mediatypes=pd.read_sql_query("""SELECT MediaTypeID, Name
FROM media_types;
 """, con=engine.connect())
df_mediatypes.head()
#dim_genre
df_genres=pd.read_sql_query("""SELECT GenreID, Name
FROM genres;
 """, con=engine.connect())
df_genres.head()
#dim_playlists
df_playlists=pd.read_sql_query("""SELECT PlaylistID, Name
FROM playlists;
 """, con=engine.connect())
df_playlists.head()
#dim_artists
df_artists=pd.read_sql_query("""SELECT ArtistID, Name
FROM artists;
 """, con=engine.connect())
df_artists.head()
#dim_invoice
df_invoices=pd.read_sql_query("""SELECT InvoiceID, Total
FROM invoices;
 """, con=engine.connect())
df_invoices.head()
#dim_customers
df_customers=pd.read_sql_query("""SELECT CustomerId,Firstname,LastName,Company,Address,City,State,Country,PostalCode,
Phone,Fax,Email,SupportRepId
FROM customers;
 """, con=engine.connect())
df_customers.head()
#dim_tracks
df_tracks=pd.read_sql_query("""SELECT TrackId,Name,MediaTypeID,GenreId,Composer,Milliseconds,Bytes,UnitPrice
FROM tracks;
 """, con=engine.connect())
df_tracks.head()
#dim_time
df_time=pd.read_sql_query("""SELECT InvoiceDate
FROM invoices;
 """, con=engine.connect())
df_time=df_time.drop_duplicates(['InvoiceDate'], keep='first')
df_time.head()
#dim_location
df_location=pd.read_sql_query("""SELECT BillingAddress, BillingCity, BillingState, BillingCountry, BillingPostalCode
FROM invoices;
 """, con=engine.connect())
df_location=df_location.drop_duplicates(['BillingAddress'], keep='first')
df_location.head()


,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode
0,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174
1,Ullevålsveien 14,Oslo,None,Norway,0171
2,Grétrystraat 63,Brussels,None,Belgium,1000
3,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7
4,69 Salem Street,Boston,MA,USA,2113


# Load

## Funciones de carga 

In [9]:
def updateData(name_table,data_db,name_columns):
	print(name_columns)
	aux_values='?'
	aux_data=[]
	aux_columns=name_columns[0]
	aux_data.append(list(data_db[name_columns[0]]))
	entities=[]
	aux_entities=[]
	for i in range(len(name_columns)-1):
		aux_values+=',?'
		aux_columns+=','+name_columns[i+1]
		aux_data.append(list(data_db[name_columns[i+1]]))
	for i in range(len(aux_data[0])):
		#aux_entities.append(i)
		for j in range(len(aux_data)):
			aux_entities.append(aux_data[j][i])
		entities.append(aux_entities)
		aux_entities=[]
	with engineWH.connect() as con:
		for i in entities:
			#print('INSERT INTO '+name_table+'('+aux_columns+') VALUES('+aux_values+')')
			#print(i)
			con.execute('INSERT INTO '+name_table+'('+aux_columns+') VALUES('+aux_values+')', i)
	con.close()


## dim_employees 

In [33]:
updateData('dim_employees',df_employees,list(df_employees.columns.values))

['LastName', 'FirstName', 'Title', 'ReportsTo', 'BirthDate', 'HireDate', 'Address', 'City', 'State', 'Country', 'PostalCode', 'Phone', 'Fax', 'Email']


## dim_albums

In [68]:
updateData('dim_albums',df_albums,list(df_albums.columns.values))

['AlbumId', 'Title']


## dim_media_types

In [70]:
updateData('dim_media_types',df_mediatypes,list(df_mediatypes.columns.values))

['MediaTypeId', 'Name']


## dim_genres

In [71]:
updateData('dim_genres',df_genres,list(df_genres.columns.values))

['GenreId', 'Name']


## dim_playlists

In [72]:
updateData('dim_playlists',df_playlists,list(df_playlists.columns.values))

['PlaylistId', 'Name']


## dim_artists

In [73]:
updateData('dim_artists',df_artists,list(df_artists.columns.values))

['ArtistId', 'Name']


## dim_invoice

In [74]:
updateData('dim_invoice',df_invoices,list(df_invoices.columns.values))

['InvoiceId', 'Total']


## dim_customers

In [76]:
updateData('dim_customers',df_customers,list(df_customers.columns.values))

['CustomerId', 'FirstName', 'LastName', 'Company', 'Address', 'City', 'State', 'Country', 'PostalCode', 'Phone', 'Fax', 'Email', 'SupportRepId']


## dim_tracks

In [ ]:
updateData('dim_tracks',df_tracks,list(df_tracks.columns.values))

['TrackId', 'Name', 'MediaTypeId', 'GenreId', 'Composer', 'Milliseconds', 'Bytes', 'UnitPrice']


## dim_time

In [8]:
updateData('dim_time',df_time,list(df_time.columns.values))

['InvoiceDate']


## dim_location

In [12]:
updateData('dim_location',df_location,list(df_location.columns.values))

['BillingAddress', 'BillingCity', 'BillingState', 'BillingCountry', 'BillingPostalCode']


# Fact_invoice_item

In [69]:

#fac_invoce_items
#ColumsAux
df_AuxInvoiceId=pd.read_sql_query("""SELECT InvoiceId
FROM invoice_items;
 """, con=engine.connect())
df_AuxInvoiceId.head()

df_CustInv=pd.read_sql_query("""SELECT InvoiceId, CustomerId
FROM invoices;
 """, con=engine.connect())

aux=list(df_AuxInvoiceId['InvoiceId'])
auxCustomer=[]
for i in aux:
    aux2=df_CustInv.index[df_CustInv['InvoiceId'] == i].tolist()
    auxCustomer.append(df_CustInv['CustomerId'][aux2[0]])

df_timeInv=pd.read_sql_query("""SELECT InvoiceId, InvoiceDate
FROM invoices;
 """, con=engine.connect())
aux=list(df_AuxInvoiceId['InvoiceId'])
auxTime1=[]
for i in aux:
    aux22=df_CustInv.index[df_CustInv['InvoiceId'] == i].tolist()
    auxTime1.append(df_timeInv['InvoiceDate'][aux22[0]])
df_dim_time=pd.read_sql_query("""SELECT TimeID, InvoiceDate
FROM dim_time;
 """, con=engineWH.connect())
auxTime=[]
for i in auxTime1:
    aux22=df_dim_time.index[df_dim_time['InvoiceDate'] == i].tolist()
    auxTime.append(df_dim_time['TimeID'][aux22[0]])

df_locationInv=pd.read_sql_query("""SELECT InvoiceId, BillingAddress
FROM invoices;
 """, con=engine.connect())
aux=list(df_AuxInvoiceId['InvoiceId'])
auxTime1=[]
for i in aux:
    aux22=df_locationInv.index[df_locationInv['InvoiceId'] == i].tolist()
    auxTime1.append(df_locationInv['BillingAddress'][aux22[0]])
df_dim_location=pd.read_sql_query("""SELECT LocationID, BillingAddress
FROM dim_location;
 """, con=engineWH.connect())
auxLocation=[]
for i in auxTime1:
    aux22=df_dim_location.index[df_dim_location['BillingAddress'] == i].tolist()
    auxLocation.append(df_dim_location['LocationID'][aux22[0]])
    

df_AuxTrackId=pd.read_sql_query("""SELECT TrackId
FROM invoice_items;
 """, con=engine.connect())
df_AuxTrackId.head()


df_tracksP=pd.read_sql_query("""SELECT TrackId, AlbumId
FROM tracks;
 """, con=engine.connect())
df_album=pd.read_sql_query("""SELECT AlbumId, Title, ArtistId
FROM albums;
 """, con=engine.connect())
aux=list(df_AuxTrackId['TrackId'])
auxAlbum=[]
for i in aux:
    aux22=df_tracksP.index[df_tracksP['TrackId'] == i].tolist()
    auxAlbum.append(df_tracksP['AlbumId'][aux22[0]])


df_tracksPlay=pd.read_sql_query("""SELECT PlaylistId, TrackId
FROM playlist_track;
 """, con=engine.connect())
aux=list(df_AuxTrackId['TrackId'])
auxPlaylist=[]
for i in aux:
    aux22=df_tracksPlay.index[df_tracksPlay['TrackId'] == i].tolist()
    auxPlaylist.append(df_tracksPlay['PlaylistId'][aux22[0]])    

    
    df_tracksPlay=pd.read_sql_query("""SELECT PlaylistId, TrackId
FROM playlist_track;
 """, con=engine.connect())
aux=list(df_AuxTrackId['TrackId'])
auxArtist=[]
for i in auxAlbum:
    aux22=df_album.index[df_album['AlbumId'] == i].tolist()
    auxArtist.append(df_album['ArtistId'][aux22[0]])

df_Quantity=pd.read_sql_query("""SELECT Quantity
FROM invoice_items;
 """, con=engine.connect())
df_Quantity.head()

,Quantity
0,1
1,1
2,1
3,1
4,1


# Fact_invoice_items

In [ ]:
columnas=['InvoiceID','CustomerID','TimeID','LocationID','TrackID','PlaylistID','ArtistID','AlbumID','Quantity']
df = pd.DataFrame(list(zip(list(df_AuxInvoiceId['InvoiceId']),auxCustomer,auxTime,auxLocation,
                          list(df_AuxTrackId['TrackId']),auxPlaylist,auxArtist,auxAlbum,
                           list(df_Quantity['Quantity']))), columns = columnas)
updateData('Fact_invoice_item',df,list(df.columns.values))

['InvoiceID', 'CustomerID', 'TimeID', 'LocationID', 'TrackID', 'PlaylistID', 'ArtistID', 'AlbumID', 'Quantity']
